In [17]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
path_indoor_distance = "Datasets/34B-IndoorDistance.csv"
path_indoor_nodistance = "Datasets/34B-IndoorNoDistance.csv"

stop_words = set(stopwords.words('english'))

In [19]:
def load_data(file_path):
    t = pd.read_csv(file_path)
    return t

def similarity_nltk(t):
    r = pd.DataFrame(columns=['Name', 'Similarity_1', 'Similarity_2', 'Similarity_3', "Average"])
    for index, row in t.iterrows():
        sentences = [row['Answer_1'], row['Answer_2'], row['Answer_3']]
        query = row['Expected_answer']
        
        tokenized_sentences = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]
        tokenized_sentences = [[word for word in sentence if word not in stop_words] for sentence in tokenized_sentences]
        tokenized_query = nltk.word_tokenize(query.lower())
        tokenized_query = [word for word in tokenized_query if word not in stop_words]
        tagged_sentences = [TaggedDocument(words=sentence, tags=[i]) for i, sentence in enumerate(tokenized_sentences)]
        
        max_epochs = 100
        vec_size = 10
        alpha = 0.005
        m = Doc2Vec(vector_size=vec_size,alpha=alpha,min_alpha=0.00025,min_count=1,dm =1)
        m.build_vocab(tagged_sentences)
        for epoch in range(max_epochs):
            #print('iteration {0}'.format(epoch))
            m.train(tagged_sentences,total_examples=m.corpus_count,epochs=m.epochs)
            m.alpha -= 0.0002
            m.min_alpha = m.alpha

        query_vec = m.infer_vector(tokenized_query)
        sentence_vecs = [m.dv[i] for i in range(len(sentences))]
        similarities = cosine_similarity([query_vec], sentence_vecs)
        r.loc[index] = [row["Name"], similarities[0][0], similarities[0][1], similarities[0][2], my_mean(similarities)]
    return r

def my_mean(s):
    return (s[0][0] + s[0][1] + s[0][2])/3 

In [20]:
d1 = load_data(path_indoor_distance)

In [21]:
result_nltk = similarity_nltk(d1)
result_nltk

,Name,Similarity_1,Similarity_2,Similarity_3,Average
0,IYD_0,0.986927,0.987201,0.985248,0.986459
1,IYD_1,0.966451,0.960747,0.958815,0.962004
2,IYD_2,0.917935,0.924941,0.917673,0.920183
3,IYD_3,0.809241,0.816884,0.810377,0.812167
4,IYD_4,0.842260,0.844779,0.867817,0.851618
5,IYD_5,0.986393,0.984638,0.986133,0.985721
6,IYD_P0,0.302249,0.311547,0.290358,0.301385
7,IYD_P1,0.995294,0.995800,0.995310,0.995468
8,IYD_P2,0.997446,0.997724,0.996914,0.997361
